# Sentiment classification with LSTM
In this notebook we will use LSTMs to do sentiment classification on the [imdb dataset](http://ai.stanford.edu/~amaas/data/sentiment/). 

In [1]:
import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 

## Dataset

To get the data: <br>
`wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz`

In [2]:
def unpack_dataset():
    ! mkdir -p data/aclImdb
    ! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    ! tar -zxvf aclImdb_v1.tar.gz -C data

In [3]:
#unpack_dataset()

In [4]:
from pathlib import Path
PATH = Path("data/aclImdb/")
list(PATH.iterdir())

[PosixPath('data/aclImdb/imdbEr.txt'),
 PosixPath('data/aclImdb/imdb.vocab'),
 PosixPath('data/aclImdb/train'),
 PosixPath('data/aclImdb/test'),
 PosixPath('data/aclImdb/README')]

In [5]:
path = PATH/"train/pos/0_9.txt"
path.read_text()

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

## Tokenization

In [6]:
# first time run this
#!python3 -m spacy download en

In [7]:
re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

my_tok = spacy.load('en')
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(sub_br(x))]

In [8]:
path = PATH/"train/pos/0_9.txt"
spacy_tok(path.read_text())[:10]

['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at']

### Computing vocab2index

In [9]:
pos_files = list((PATH/"train"/"pos").iterdir())
neg_files = list((PATH/"train"/"neg").iterdir())
all_files = pos_files + neg_files
all_files[:5]

[PosixPath('data/aclImdb/train/pos/10544_8.txt'),
 PosixPath('data/aclImdb/train/pos/9530_9.txt'),
 PosixPath('data/aclImdb/train/pos/9901_8.txt'),
 PosixPath('data/aclImdb/train/pos/11951_8.txt'),
 PosixPath('data/aclImdb/train/pos/7441_7.txt')]

In [10]:
# takes some time
counts = Counter()
for path in all_files:
    counts.update(spacy_tok(path.read_text()))

In [11]:
#counts

In [12]:
len(counts.keys())

103688

In [13]:
for word in list(counts):
    if counts[word] < 5:
        del counts[word]

In [14]:
len(counts.keys())

33914

In [15]:
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [16]:
#vocab2index

## Dataset

In [17]:
# spacy_tok takes a while. Run it just once
def encode_sentence(path, vocab2index, N=400, padding_start=True):
    x = spacy_tok(path.read_text())
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])
    l = min(N, len(enc1))
    if padding_start:
        enc[:l] = enc1[:l]
    else:
        enc[N-l:] = enc1[:l]
    return enc, l

In [18]:
path = PATH/"train/neg/211_4.txt"
encode_sentence(path, vocab2index, N=400, padding_start=False)

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            1,  1782,     4,  2723,     4,    29,    36,    37,    15,
         2388,   320,     6,    23,   351,     8,   316,   254,   680,
         1382,   103,   334,  7723,   103, 12825,     3,   369,   951,
            3, 16948,    19,  1791,    55,  1573,    36,     1,   153,
      

In [19]:
class ImdbDataset(Dataset):
    def __init__(self, PATH, train="train", N=400, padding_start=True):
        self.path_to_images = PATH/train
        self.pos_files = list((self.path_to_images/"pos").iterdir())
        self.neg_files = list((self.path_to_images/"neg").iterdir())
        self.files = self.pos_files + self.neg_files
        # pos 1, neg 0
        self.y = np.concatenate((np.ones(len(self.pos_files), dtype=int),
                                np.zeros(len(self.neg_files), dtype=int)), axis=0)
        # it is important to run encode_sentence in the init
        self.X = [encode_sentence(path, vocab2index, N, padding_start) for path in self.files]
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x, s = self.X[idx]
        return x, s, self.y[idx]

In [20]:
train_ds_v0 = ImdbDataset(PATH, padding_start=False)
valid_ds_v0 = ImdbDataset(PATH, "test", padding_start=False)

In [21]:
batch_size = 1000
train_dl_v0 = DataLoader(train_ds_v0, batch_size=batch_size, shuffle=True)
valid_dl_v0 = DataLoader(valid_ds_v0, batch_size=batch_size)

In [22]:
train_ds_v0[1]

(array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

## Understanding LSTMs 

nn.LSTM() <br>
`input_size` – The dimension of the embedding for each word <br>
`hidden_size` – The number of features in the hidden state h <br>
`num_layers` – Number of recurrent layers <br>
`batch_first` – If True, then the input and output tensors are provided as (batch, seq, feature) <br>

In [23]:
inputs = [torch.randn(1, 2) for _ in range(5)] # make a sequence of length 5
inputs = torch.cat(inputs).view(1, len(inputs), -1)
inputs

tensor([[[-0.5992,  0.4123],
         [ 1.8419, -0.9393],
         [ 0.1464,  0.1443],
         [-0.1934, -1.0094],
         [-0.5510, -1.0726]]])

In [24]:
lstm = nn.LSTM(input_size=2, hidden_size=4, batch_first=True)

In [25]:
out, (hidden, cell) = lstm(inputs)

In [26]:
out.shape

torch.Size([1, 5, 4])

In [27]:
hidden.shape

torch.Size([1, 1, 4])

## LSTM V0 model

In [28]:
class LSTMV0Model(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(LSTMV0Model,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        out_pack, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [29]:
def train_epocs_v0(model, train_dl, val_dl, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            # s is not used in this model
            x = x.long().cuda()
            y = y.float().cuda()
            y_pred = model(x)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics_v0(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [30]:
def val_metrics_v0(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in valid_dl:
        # s is not used here
        x = x.long().cuda()
        y = y.float().unsqueeze(1).cuda()
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [31]:
batch_size = 5000
train_dl = DataLoader(train_ds_v0, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds_v0, batch_size=batch_size)

In [32]:
vocab_size = len(words)
print(vocab_size)
model = LSTMV0Model(vocab_size, 50, 50).cuda()

33916


In [33]:
train_epocs_v0(model, train_dl, valid_dl, epochs=30, lr=0.01)

train loss 0.681 val loss 0.668 and val accuracy 0.586
train loss 0.579 val loss 0.612 and val accuracy 0.710
train loss 0.462 val loss 0.547 and val accuracy 0.763
train loss 0.351 val loss 0.648 and val accuracy 0.759
train loss 0.299 val loss 0.529 and val accuracy 0.814
train loss 0.228 val loss 0.504 and val accuracy 0.839


In [34]:
train_epocs_v0(model, train_dl, valid_dl, epochs=30, lr=0.001)

train loss 0.190 val loss 0.534 and val accuracy 0.845
train loss 0.178 val loss 0.546 and val accuracy 0.846
train loss 0.174 val loss 0.581 and val accuracy 0.842
train loss 0.170 val loss 0.579 and val accuracy 0.843
train loss 0.164 val loss 0.624 and val accuracy 0.839
train loss 0.161 val loss 0.616 and val accuracy 0.840


## GRU Model 
GRU have less parameters but often have as good or better performance than an LSTM

In [35]:
class GRUV0Model(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(GRUV0Model,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        out_pack, ht = self.gru(x)
        return self.linear(ht[-1])

In [36]:
vocab_size = len(words)
model = GRUV0Model(vocab_size, 50, 50).cuda()

In [37]:
train_epocs_v0(model, train_dl, valid_dl, epochs=30, lr=0.01)

train loss 0.684 val loss 0.675 and val accuracy 0.582
train loss 0.569 val loss 0.551 and val accuracy 0.733
train loss 0.450 val loss 0.524 and val accuracy 0.762
train loss 0.316 val loss 0.579 and val accuracy 0.779
train loss 0.219 val loss 0.521 and val accuracy 0.828
train loss 0.156 val loss 0.526 and val accuracy 0.842


In [38]:
train_epocs_v0(model, train_dl, valid_dl, epochs=30, lr=0.001)

train loss 0.120 val loss 0.494 and val accuracy 0.859
train loss 0.114 val loss 0.497 and val accuracy 0.861
train loss 0.108 val loss 0.483 and val accuracy 0.867
train loss 0.105 val loss 0.482 and val accuracy 0.868
train loss 0.103 val loss 0.494 and val accuracy 0.867
train loss 0.121 val loss 0.411 and val accuracy 0.869


## Model with variable length
dynamic padding + pack_padded_sequence

`pack_padded_sequence` packs a Tensor containing padded sequences of variable length.

In [39]:
def encode_sentence_no_padding(path, vocab2index):
    x = spacy_tok(path.read_text())
    return np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])

In [40]:
path = PATH/"train/neg/211_4.txt"
#encode_sentence_no_padding(path, vocab2index)

In [41]:
class ImdbDataset2(Dataset):
    def __init__(self, PATH, train="train"):
        self.path_to_images = PATH/train
        self.pos_files = list((self.path_to_images/"pos").iterdir())
        self.neg_files = list((self.path_to_images/"neg").iterdir())
        self.files = self.pos_files + self.neg_files
        # pos 1, neg 0
        self.y = np.concatenate((np.ones(len(self.pos_files), dtype=int),
                                np.zeros(len(self.neg_files), dtype=int)), axis=0)
        # it is important to run encode_sentence in the init
        self.X = [encode_sentence_no_padding(path, vocab2index) for path in self.files]
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.X[idx]
        return x, self.y[idx]

In [42]:
train_ds = ImdbDataset2(PATH)
valid_ds = ImdbDataset2(PATH, "test")

In [43]:
#train_ds[0]

### collate_fn function
The `collate_fn` merges a list of samples to form a mini-batch. It is an optional parameter to our data loader.

In [45]:
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (sentences, labels).
    
    Need custom collate_fn because merging sequences (including padding) is not 
    supported in default. Sequences are padded to the maximum length of mini-batch 
    sequences (dynamic padding).
    
    Args:
        data: list of tuple (sentence, label). 
            - list of word indices of variable length
            - label, 0 or 1
    Returns:
        packed_batch: (PackedSequence), see torch.nn.utils.rnn.pack_padded_sequence
        sencences: torch tensor of shape (batch_size, max_len).
        labels: torch tensor of shape (batch_size, 1).
        lengths: list; valid length for each padded sentence. 
    """
    # Sort a data list by sentences length (descending order).
    data.sort(key=lambda x: len(x[0]), reverse=True)
    sentences, labels = zip(*data)
    
    # stack labels
    labels = torch.Tensor(labels)
    
    # Merge sentences
    lengths = [len(s) for s in sentences]
   
    sents = torch.zeros(len(sentences), max(lengths)).long()
    for i, s in enumerate(sentences):
        end = lengths[i]
        sents[i, :end] = torch.Tensor(s[:end])        
    
    return sents, lengths, labels

In [46]:
# tiny example of our data
data = [([4, 545, 23, 1], 0), ([34, 84], 1), ([23, 6, 774], 0)]

In [47]:
# note how the data ir reorder and padded
collate_fn(data)

(tensor([[  4, 545,  23,   1],
         [ 23,   6, 774,   0],
         [ 34,  84,   0,   0]]),
 [4, 3, 2],
 tensor([0., 0., 1.]))

In [48]:
# more realistic example
data = [train_ds[0], train_ds[1], train_ds[2]]

In [49]:
collate_fn(data)

(tensor([[156,  12, 262,  ..., 271, 281, 214],
         [  2,   3,   4,  ...,   0,   0,   0],
         [181, 182, 183,  ...,   0,   0,   0]]),
 [710, 359, 188],
 tensor([1., 1., 1.]))

In [50]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, collate_fn=collate_fn)

In [51]:
sents, lengths, labels = next(iter(train_dl))

In [52]:
sents.shape, labels.shape

(torch.Size([5, 982]), torch.Size([5]))

In [53]:
lengths

[982, 547, 240, 204, 141]

In [54]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(GRUModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x, lengths):
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [55]:
def train_epocs(model, train_dl, valid_dl, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long().cuda()
            y = y.float().cuda()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [56]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in valid_dl:
        x = x.long().cuda()
        y = y.float().unsqueeze(1).cuda()
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [57]:
batch_size = 3000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, collate_fn=collate_fn)

In [58]:
vocab_size = len(words)
model = GRUModel(vocab_size, 50, 50).cuda()

In [59]:
train_epocs(model, train_dl, valid_dl, epochs=30, lr=0.01)

train loss 0.663 val loss 0.638 and val accuracy 0.637
train loss 0.461 val loss 0.558 and val accuracy 0.774
train loss 0.255 val loss 0.404 and val accuracy 0.861
train loss 0.159 val loss 0.446 and val accuracy 0.871
train loss 0.101 val loss 0.525 and val accuracy 0.874
train loss 0.074 val loss 0.567 and val accuracy 0.874


In [60]:
train_epocs(model, train_dl, valid_dl, epochs=30, lr=0.001)

train loss 0.055 val loss 0.590 and val accuracy 0.876
train loss 0.050 val loss 0.625 and val accuracy 0.875
train loss 0.047 val loss 0.678 and val accuracy 0.871
train loss 0.042 val loss 0.697 and val accuracy 0.871
train loss 0.042 val loss 0.703 and val accuracy 0.871
train loss 0.040 val loss 0.710 and val accuracy 0.872


## Reordering in the forward function

In [62]:
# back to padding
train_ds = ImdbDataset(PATH, padding_start=True)
valid_ds= ImdbDataset(PATH, "test", padding_start=True)

batch_size = 7
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

x, s, y = next(iter(train_dl)) # here s is the length of the sentences

In [63]:
x.shape, s.shape

(torch.Size([7, 400]), torch.Size([7]))

In [64]:
s

tensor([400, 329, 154, 200, 223, 316, 265])

In [65]:
y

tensor([1, 1, 1, 0, 1, 0, 1])

In [66]:
# sort by length so we can use pack_padded_sequence
s, index = s.sort(0, descending=True)
x = x[index]

In [67]:
s

tensor([400, 329, 316, 265, 223, 200, 154])

In [68]:
index

tensor([0, 1, 5, 6, 4, 3, 2])

In [69]:
y[index]

tensor([1, 1, 0, 1, 1, 0, 1])

In [70]:
vocab_size = len(words)
embedding_dim = 10
embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

In [71]:
x = embed(x.long())
x.shape

torch.Size([7, 400, 10])

In [72]:
hidden_dim = 9
lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

In [73]:
# RNN will not perform calculation on pad elements if pack_padded_sequence is used
x_pack = pack_padded_sequence(x, list(s), batch_first=True)

In [74]:
out_pack, (ht, ct) = lstm(x_pack)

In [75]:
## final hidden layer
ht.shape

torch.Size([1, 7, 9])

In [76]:
ht[-1].shape

torch.Size([7, 9])

In [77]:
linear = nn.Linear(hidden_dim, 1)
y_hat = linear(ht[-1])
y_hat

tensor([[-0.3660],
        [-0.2679],
        [-0.2597],
        [-0.3598],
        [-0.3735],
        [-0.2724],
        [-0.3357]], grad_fn=<AddmmBackward>)

In [78]:
index.unsqueeze(1).shape

torch.Size([7, 1])

In [79]:
# takes back to the original ordering
h = torch.zeros_like(y_hat).scatter_(0, index.unsqueeze(1), y_hat)

In [80]:
h

tensor([[-0.3660],
        [-0.2679],
        [-0.3357],
        [-0.2724],
        [-0.3735],
        [-0.2597],
        [-0.3598]], grad_fn=<ScatterBackward0>)

In [81]:
y_hat

tensor([[-0.3660],
        [-0.2679],
        [-0.2597],
        [-0.3598],
        [-0.3735],
        [-0.2724],
        [-0.3357]], grad_fn=<AddmmBackward>)

In [82]:
index

tensor([0, 1, 5, 6, 4, 3, 2])

### Model with sorting in the forward

In [83]:
class LSTMModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(LSTMModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.5)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        
    def forward(self, x, s):
        # sorting
        s, sort_index = torch.sort(s, 0,descending=True)
        s = s.numpy().tolist()
        x = x[sort_index]
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return torch.zeros_like(out).scatter_(0, sort_index.unsqueeze(1).cuda(), out) 

In [84]:
batch_size = 2000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [85]:
vocab_size = len(words)
print(vocab_size)
model = LSTMModel(vocab_size, 50, 50).cuda()

33916


In [87]:
train_epocs(model, train_dl, valid_dl, epochs=30, lr=0.01)

train loss 0.670 val loss 0.661 and val accuracy 0.608
train loss 0.511 val loss 0.594 and val accuracy 0.758
train loss 0.414 val loss 0.493 and val accuracy 0.770
train loss 0.249 val loss 0.675 and val accuracy 0.798
train loss 0.223 val loss 0.448 and val accuracy 0.850
train loss 0.136 val loss 0.576 and val accuracy 0.849


In [88]:
train_epocs(model, train_dl, valid_dl, epochs=30, lr=0.001)

train loss 0.105 val loss 0.622 and val accuracy 0.849
train loss 0.100 val loss 0.648 and val accuracy 0.849
train loss 0.095 val loss 0.670 and val accuracy 0.847
train loss 0.089 val loss 0.673 and val accuracy 0.851
train loss 0.084 val loss 0.670 and val accuracy 0.850
train loss 0.086 val loss 0.672 and val accuracy 0.852


In [89]:
train_epocs(model, train_dl, valid_dl, epochs=30, lr=0.001)

train loss 0.080 val loss 0.699 and val accuracy 0.850
train loss 0.076 val loss 0.710 and val accuracy 0.851
train loss 0.074 val loss 0.735 and val accuracy 0.848
train loss 0.074 val loss 0.689 and val accuracy 0.854
train loss 0.068 val loss 0.719 and val accuracy 0.851
train loss 0.068 val loss 0.701 and val accuracy 0.855


In [92]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [93]:
! mkdir $PATH/"models"

mkdir: cannot create directory ‘data/aclImdb/models’: File exists


In [94]:
p = PATH/"models/model-855.pth"
save_model(model, p)

In [95]:
val_metrics(model, valid_dl)

(0.7453345537185669, tensor(0.8508, device='cuda:0'))

In [96]:
load_model(model, p)

## GRU model with dropout

In [97]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(GRUModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(0.5)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        
    def forward(self, x, s):
        s, sort_index = torch.sort(s, 0,descending=True)
        s = s.numpy().tolist()
        x = x[sort_index]
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, list(s), batch_first=True)
        out_pack, ht= self.gru(x_pack)
        out = self.linear(ht[-1])
        return torch.zeros_like(out).scatter_(0, sort_index.unsqueeze(1).cuda(), out)

## Bidirectional and multiple layers GRUs / LSTMs

In [100]:
batch_size = 7
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

x,s,y = next(iter(train_dl)) # here s is the length of the sentences

In [101]:
vocab_size = len(words)
embedding_dim = 10
hidden_dim = 9
embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
lstm1 = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
lstm2 = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)

In [102]:
s, index = s.sort(0, descending=True)
x = x[index]
x = embed(x.long())
x_pack = pack_padded_sequence(x, list(s), batch_first=True)

In [103]:
lstm_out, (ht, ct) = lstm1(x)

In [104]:
ht.shape

torch.Size([2, 7, 9])

In [105]:
ht[-2,:,:].shape

torch.Size([7, 9])

In [106]:
lstm_out, (ht2, ct2) = lstm2(x)

In [107]:
ht2.shape

torch.Size([4, 7, 9])

In [108]:
ht2[-2,:,:].shape, ht2[-1,:,:].shape

(torch.Size([7, 9]), torch.Size([7, 9]))

In [109]:
#concat the final forward (ht[-2,:,:]) and backward (ht[-1,:,:]) hidden layers      
h = torch.cat((ht2[-2,:,:], ht2[-1,:,:]), dim = 1)
h.shape

torch.Size([7, 18])

In [110]:
# This is slow

In [133]:
class BiGRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(BiGRUModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True,
                          dropout=0.3, bidirectional=True)
        
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(2*hidden_dim, 1)
        
    def forward(self, x, s):
        s, sort_index = torch.sort(s, 0,descending=True)
        s = s.numpy().tolist()
        x = x[sort_index]
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True)
        out_pack, ht = self.gru(x_pack)
        h = torch.cat((ht[-2,:,:], ht[-1,:,:]), dim = 1)
        h = self.linear(h)
        return torch.zeros_like(h).scatter_(0, sort_index.unsqueeze(1).cuda(), h)

In [134]:
vocab_size = len(words)
model = BiGRUModel(vocab_size, 50, 50).cuda()

In [135]:
batch_size = 2000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [136]:
train_epocs(model, train_dl, valid_dl, epochs=30, lr=0.01)

train loss 0.645 val loss 0.626 and val accuracy 0.676
train loss 0.290 val loss 0.359 and val accuracy 0.865
train loss 0.153 val loss 0.359 and val accuracy 0.887
train loss 0.094 val loss 0.456 and val accuracy 0.875
train loss 0.058 val loss 0.540 and val accuracy 0.876
train loss 0.034 val loss 0.611 and val accuracy 0.875


In [137]:
train_epocs(model, train_dl, valid_dl, epochs=30, lr=0.001)

train loss 0.027 val loss 0.664 and val accuracy 0.873
train loss 0.025 val loss 0.678 and val accuracy 0.873
train loss 0.021 val loss 0.710 and val accuracy 0.871
train loss 0.021 val loss 0.694 and val accuracy 0.874
train loss 0.018 val loss 0.722 and val accuracy 0.873
train loss 0.019 val loss 0.721 and val accuracy 0.873


In [138]:
vocab_size = len(words)
model = BiGRUModel(vocab_size, 50, 50).cuda()

In [ ]:
train_epocs(model, train_dl, valid_dl, epochs=30, lr=0.005)

train loss 0.661 val loss 0.613 and val accuracy 0.666
train loss 0.404 val loss 0.390 and val accuracy 0.833
train loss 0.256 val loss 0.372 and val accuracy 0.862
train loss 0.177 val loss 0.339 and val accuracy 0.883
train loss 0.134 val loss 0.399 and val accuracy 0.881


In [ ]:
train_epocs(model, train_dl, valid_dl, epochs=30, lr=0.001)

## References

The model in this notebook is adapted from this [pytorch tutorial](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html). 